In [3]:
import numpy as np
import qec
import pymatching, stim
import sinter

In [9]:

def ghz(d, eta, order, num_qubits=3, measure='x'):
    assert measure in ['x', 'z']
    print(f"order = {order}")
    p = 0.001
    logical_qubits = [qec.surface_code.RotatedSurfaceCode(d, d) for _ in range(num_qubits)]
    lc = qec.LogicalCircuit(logical_qubits, initialize_circuit=False,
                            loss_noise_scale_factor=0.0, spam_noise_scale_factor=1,
                            gate_noise_scale_factor=1, idle_noise_scale_factor=0,
                            measurement_error_rate=1e-7,
                            reset_error_rate=0,
                            entangling_error_rate=(eta * p / (1+eta) / 2, eta * p / (1+eta) / 2, eta * p / (1+eta)),
                            single_qubit_error_rate=(0, 0, 0))

    # First prepare all logical qubits in |0>
    lc.append(qec.surface_code.prepare_zero, list(range(0, len(logical_qubits))))

    # Rotate noiselessly
    lc.append(qec.surface_code.rotate_code, 0)

    # Hadamard all qubits
    lc.append(qec.surface_code.global_h, list(range(len(logical_qubits))), move_duration=0)

    for _ in range(len(order)):
        lc.append(qec.surface_code.global_cz, [order[_][0], order[_][1]], move_duration=0)
        lc.append(qec.surface_code.global_h, order[_][1], move_duration=0)

    lc.append_from_stim_program_text("""TICK""") # starting a QEC round
    lc.append(qec.surface_code.measure_stabilizers, list(range(len(logical_qubits))), order='fowler', with_cnot=True, compare_with_previous=True) # append QEC rounds
    lc.append_from_stim_program_text("""TICK""") # starting a QEC round
    
    if measure == 'x':
        lc.append(qec.surface_code.measure_x, list(range(len(logical_qubits))), observable_include=False)
        lc.append('MOVE_TO_NO_NOISE', lc.qubit_indices, 0)
        global_x = []
        for index in range(len(lc.logical_qubits)):
            global_x += lc.logical_qubits[index].logical_x_operator + [stim.GateTarget(stim.target_combiner())]
        lc.append('MPP', global_x[:-1])
        lc.append('OBSERVABLE_INCLUDE', [stim.target_rec(-1)], 0)

    elif measure == 'z':
        lc.append(qec.surface_code.measure_z, list(range(len(logical_qubits))), observable_include=False)
        lc.append('MOVE_TO_NO_NOISE', lc.qubit_indices, 0)

        for index in range(len(logical_qubits) - 1):
            lc.append('MPP', lc.logical_qubits[index].logical_z_operator + [stim.GateTarget(stim.target_combiner())] +
                    lc.logical_qubits[index + 1].logical_z_operator)
            lc.append('OBSERVABLE_INCLUDE', [stim.target_rec(-1)], lc.num_observables)

    return lc


In [5]:





#biases = 10**np.linspace(0, 2., 10)
d = 3

z_fidelities_1 = []
x_fidelities_1 = []
z_fidelities_2 = []
x_fidelities_2 = []

for n in [3]:
    order_1 = []
    order_2 = []
    for i in range(1, n):
        order_1.append((0, i))
        order_2.append((i - 1, i))

    orders = [order_1, order_2]
    print('GHZ state of {} logical qubits with d = {}'.format(n, d))
    for (o, order) in enumerate(orders):
        for bias in [1/2]:
            for measure in ['x', 'z']:
                num_shots = 10000
                lc = ghz(d, bias, order, num_qubits=n, measure=measure)
                print(lc)
                dem = lc.without_loss().detector_error_model(decompose_errors=False, approximate_disjoint_errors=True)

                # Sample the circuit.
                shots = lc.compile_sampler().sample(num_shots)
                detector_shots, observable_shots = lc.without_loss().compile_m2d_converter().convert(measurements=shots,
                                                                                                    separate_observables=True)

                detector_error_model = lc.detector_error_model(decompose_errors=True, approximate_disjoint_errors=True, ignore_decomposition_failures=True, allow_gauge_detectors=False)
                            
                prediction = sinter.predict_observables(
                                    dem=detector_error_model,
                                    dets=detector_shots,
                                    decoder='pymatching',)
                # prediction = qec.correlated_decoders.mle.decode_gurobi_with_dem(dem, detector_shots)
                corrected_observables = 1 - np.logical_xor(observable_shots, prediction)
                if measure == 'x' and o == 0:
                    x_fidelities_1.append(np.mean(corrected_observables))
                elif measure == 'x' and o == 1:
                    x_fidelities_2.append(np.mean(corrected_observables))
                elif measure == 'z' and o == 0:
                    z_fidelities_1.append(np.mean(np.alltrue(corrected_observables == np.ones(n-1), axis=1)))
                elif measure == 'z' and o == 1:
                    z_fidelities_2.append(np.mean(np.alltrue(corrected_observables == np.ones(n-1), axis=1)))


    print(x_fidelities_1)
    print(x_fidelities_2)
    print(z_fidelities_1)
    print(z_fidelities_2)


GHZ state of 3 logical qubits with d = 3
R 1 2 3 7 8 9 13 14 15 18 19 20 24 25 26 30 31 32 35 36 37 41 42 43 47 48 49 0 5 11 16 17 22 28 33 34 39 45 50 4 6 10 12 21 23 27 29 38 40 44 46
X_ERROR(0) 1 2 3 4 6 7 8 9 10 12 13 14 15 18 19 20 21 23 24 25 26 27 29 30 31 32 35 36 37 38 40 41 42 43 44 46 47 48 49
M 0 5 11 16 17 22 28 33 34 39 45 50
H 1 2 3 7 8 9 13 14 15 18 19 20 24 25 26 30 31 32 35 36 37 41 42 43 47 48 49 4 6 10 12 21 23 27 29 38 40 44 46
PAULI_CHANNEL_1(0, 0, 0) 1 2 3 7 8 9 13 14 15 18 19 20 24 25 26 30 31 32 35 36 37 41 42 43 47 48 49 4 6 10 12 21 23 27 29 38 40 44 46
CZ 4 2 10 7 12 9 21 19 27 24 29 26 38 36 44 41 46 43
PAULI_CHANNEL_1(0.000166667, 0.000166667, 0.000333333) 1 2 3 4 6 7 8 9 10 12 13 14 15 18 19 20 21 23 24 25 26 27 29 30 31 32 35 36 37 38 40 41 42 43 44 46 47 48 49
TICK
PAULI_CHANNEL_1(0, 0, 0) 1 2 3 4 6 7 8 9 10 12 13 14 15 18 19 20 21 23 24 25 26 27 29 30 31 32 35 36 37 38 40 41 42 43 44 46 47 48 49
CZ 4 1 6 3 12 8 21 18 23 20 29 25 38 35 40 37 46 42
PAULI

In [10]:





#biases = 10**np.linspace(0, 2., 10)
d = 3

z_fidelities_1 = []
x_fidelities_1 = []
z_fidelities_2 = []
x_fidelities_2 = []

for n in [3]:
    order_1 = []
    order_2 = []
    for i in range(1, n):
        order_1.append((0, i))
        order_2.append((i - 1, i))

    orders = [order_1]
    print('GHZ state of {} logical qubits with d = {}'.format(n, d))
    for (o, order) in enumerate(orders):
        for bias in [1/2]:
            for measure in ['z']:
                num_shots = 10000
                lc = ghz(d, bias, order, num_qubits=n, measure=measure)
                print(lc)
                dem = lc.without_loss().detector_error_model(decompose_errors=False, approximate_disjoint_errors=True)

                # Sample the circuit.
                shots = lc.compile_sampler().sample(num_shots)
                detector_shots, observable_shots = lc.without_loss().compile_m2d_converter().convert(measurements=shots,
                                                                                                    separate_observables=True)

                detector_error_model = lc.detector_error_model(decompose_errors=True, approximate_disjoint_errors=True, ignore_decomposition_failures=True, allow_gauge_detectors=False)
                            
                prediction = sinter.predict_observables(
                                    dem=detector_error_model,
                                    dets=detector_shots,
                                    decoder='pymatching',)
                # prediction = qec.correlated_decoders.mle.decode_gurobi_with_dem(dem, detector_shots)
                corrected_observables = 1 - np.logical_xor(observable_shots, prediction)
                if measure == 'x' and o == 0:
                    x_fidelities_1.append(np.mean(corrected_observables))
                elif measure == 'x' and o == 1:
                    x_fidelities_2.append(np.mean(corrected_observables))
                elif measure == 'z' and o == 0:
                    z_fidelities_1.append(np.mean(np.alltrue(corrected_observables == np.ones(n-1), axis=1)))
                elif measure == 'z' and o == 1:
                    z_fidelities_2.append(np.mean(np.alltrue(corrected_observables == np.ones(n-1), axis=1)))


    print(x_fidelities_1)
    print(x_fidelities_2)
    print(z_fidelities_1)
    print(z_fidelities_2)


GHZ state of 3 logical qubits with d = 3
order = [(0, 1), (0, 2)]
R 1 2 3 7 8 9 13 14 15 18 19 20 24 25 26 30 31 32 35 36 37 41 42 43 47 48 49 0 5 11 16 17 22 28 33 34 39 45 50 4 6 10 12 21 23 27 29 38 40 44 46
X_ERROR(0) 1 2 3 4 6 7 8 9 10 12 13 14 15 18 19 20 21 23 24 25 26 27 29 30 31 32 35 36 37 38 40 41 42 43 44 46 47 48 49
M 0 5 11 16 17 22 28 33 34 39 45 50
H 1 2 3 7 8 9 13 14 15 18 19 20 24 25 26 30 31 32 35 36 37 41 42 43 47 48 49 4 6 10 12 21 23 27 29 38 40 44 46
PAULI_CHANNEL_1(0, 0, 0) 1 2 3 7 8 9 13 14 15 18 19 20 24 25 26 30 31 32 35 36 37 41 42 43 47 48 49 4 6 10 12 21 23 27 29 38 40 44 46
CZ 4 2 10 7 12 9 21 19 27 24 29 26 38 36 44 41 46 43
PAULI_CHANNEL_1(0.000166667, 0.000166667, 0.000333333) 1 2 3 4 6 7 8 9 10 12 13 14 15 18 19 20 21 23 24 25 26 27 29 30 31 32 35 36 37 38 40 41 42 43 44 46 47 48 49
TICK
PAULI_CHANNEL_1(0, 0, 0) 1 2 3 4 6 7 8 9 10 12 13 14 15 18 19 20 21 23 24 25 26 27 29 30 31 32 35 36 37 38 40 41 42 43 44 46 47 48 49
CZ 4 1 6 3 12 8 21 18 23 20 29 2